!pip install nltk

In [117]:
import pandas as pd
import numpy as np
import json
import ast
from nltk.tokenize import WhitespaceTokenizer
import random
from azureml.core import Workspace, Dataset

In [114]:
# reading a dataset from the datasets registry
subscription_id = '9d0dfa04-d2f8-4521-b945-b3a7dbf43946'
resource_group = 'CougsInAzure'
workspace_name = 'CougsInAzure2'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='V2Dataset')

In [98]:
# ONLY USING FOR TESTING, READING FROM A CSV
headers =["LowerUnderscoreUpper", "HasUnderscore", "LowerUpperLower", "MultipleLowerUpperLower", "ExactlyTwoUppercase", "AllLowerMoreThan",
    "AdjacentUppers", "StartLetterEndNonLetter", "LengthGTLT", "IllegalHostnameChars", "AlphaOrDigit", "HostIllegalEnding", "ContainsPeriod",
    "HasSlash", "HasMultipleSlash", "HasPossibleExtension", "NumbersThenPeriod", "AtLeastFourDigits", "HasPeriodAndSlash", "HasInstanceNumPeriod", 
    "HasMultipleNumPeriod", "Label"]
df = pd.read_csv("~/cloudfiles/code/Users/capstonesipsml/Data/token_features.csv", names=headers, header=0)
df['Label'].value_counts()


nothing       14975
filename       8741
hostname       5025
username       3850
servername      186
Name: Label, dtype: int64

In [119]:
dataset.to_pandas_dataframe()


In [99]:
# splitting the datasets into positive and negative lists
def IsPositive(r):
    return True if r[-1] !="nothing" else False
def IsNegative(r):
    return True if r[-1] == "nothing" else False

df_list = df.values.tolist()
print(df_list[0])

positive_dataset = list(filter(IsPositive, df_list))
negative_dataset = list(filter(IsNegative, df_list))


random.shuffle(positive_dataset)
random.shuffle(negative_dataset)



[False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, 'nothing']


In [100]:
# Combining the dataset into 1
dataset = []
l = len(negative_dataset) if len(negative_dataset) < len(positive_dataset) else len(positive_dataset)

for i in range(0,l):
    dataset.append(positive_dataset[i])
    dataset.append(negative_dataset[i])
print(dataset[:5])
print(len(dataset))

[[False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, 'hostname'], [False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, 'nothing'], [False, False, False, False, False, True, False, False, False, True, False, False, True, True, True, True, False, True, False, False, False, 'filename'], [False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, 'nothing'], [False, False, False, False, False, True, False, False, False, True, False, False, True, True, True, True, False, False, False, False, False, 'filename']]
29950


In [104]:
# Splitting the dataset into train and test

X = list(map(lambda x: x[:-1], dataset))
y = list(map(lambda x: x[-1], dataset))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2)
print(len(X))


29950


In [78]:
# Functions to compute different metrics

def calcAccuracy(target, predictions):
    total = len(predictions)
    correct = 0
    for i in range(len(predictions)):
        if target[i] == predictions[i]:
            correct += 1
        accuracy = float(correct) / float(total)
    return accuracy

def FalsePositivesMultiClass(x_test, y_test, pred):
    false_positives = []
    for i in range(0, len(x_test)):
        if y_test[i] == 'nothing' and pred[i] != 'nothing':
            false_positives.append(x_test[i])
    return false_positives

def FalseNegativesMultiClass(x_test, y_test, pred):
    false_negatives = []
    for i in range(0,len(x_test)):
        if y_test[i] != 'nothing' and pred[i] == 'nothing':
            false_negatives.append(x_test[i])
    return false_negatives

def MislabeledMultiClass(x_test, y_test, pred):
    mislabeled = []
    for i in range(0,len(x_test)):
        if y_test[i] != 'nothing' and pred[i] != 'nothing' and y_test[i] != pred[i]:
            mislabeled.append(x_test[i])
    return mislabeled

def FalsePositives(x_test, y_test, pred):
    false_positives = []
    for i in range(0, len(x_test)):
        if y_test[i] == 0 and pred[i] == 1:
            false_positives.append(x_test[i])
    return false_positives

def FalseNegatives(x_test, y_test, pred):
    false_negatives = []
    for i in range(0,len(x_test)):
        if y_test[i] != 1 and pred[i] == 0:
            false_negatives.append(x_test[i])
    return false_negatives

def PrintResults(classifier, acc, recall, precision):
    print(classifier + " Classifier")
    print("Accuracy= ", acc)
    print("Precision= ", precision)
    print("Recall= ", recall)

In [85]:
# BEGIN MACHINE LEARNING MODELS
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score
from sklearn.neural_network import MLPClassifier

In [105]:
# DECISION TREE CLASSIFIER
dt = DecisionTreeClassifier().fit(X_train, y_train)
pred = dt.predict(X_test)
acc = calcAccuracy(y_test, pred)
#recall = recall_score(y_test, pred,average='macro')
#precision = precision_score(y_test, pred, average='macro')
PrintResults("Decision Tree", acc, "N/A", "N/A")

false_negatives = FalseNegativesMultiClass(X_test, y_test, pred)
false_positives = FalsePositivesMultiClass(X_test, y_test, pred)
mislabeled = MislabeledMulitiClass(X_test,y_test,pred)

{"IP","filename",'servername','username','nothing'}
print(f"\nFP-> {len(false_positives)} \nFN-> {len(false_negatives)} \nMislabeled-> {len(mislabeled)}")

Decision Tree Classifier
Accuracy=  0.9529542695265075
Precision=  N/A
Recall=  N/A

FP-> 126 
FN-> 333 
Mislabeled-> 6


In [106]:
# KNN CLASSIFIER
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train, y_train)
pred = knn.predict(X_test)
acc = calcAccuracy(y_test, pred)
#recall = recall_score(y_test, pred)
#precision = precision_score(y_test, pred)
PrintResults("KNN", acc, "N/A", "N/A")

false_negatives = FalseNegativesMultiClass(X_test, y_test, pred)
false_positives = FalsePositivesMultiClass(X_test, y_test, pred)
mislabeled = MislabeledMultiClass(X_test,y_test,pred)

print(f"\nFP-> {len(false_positives)} \nFN-> {len(false_negatives)} \n Mislabeled-> {len(mislabeled)}")

KNN Classifier
Accuracy=  0.9286726021853501
Precision=  N/A
Recall=  N/A

FP-> 439 
FN-> 260 
 Mislabeled-> 6


In [107]:
# LOGISTIC REGRESSION CLASSIFER
lr = LogisticRegression(random_state=0)
lr.fit(X_train, y_train)
pred = lr.predict(X_test)
acc = calcAccuracy(y_test, pred)
#recall = recall_score(y_test, pred)
#precision = precision_score(y_test, pred)
PrintResults("Logistic Regression", acc, "N/A", "N/A")

false_negatives = FalseNegativesMultiClass(X_test, y_test, pred)
false_positives = FalsePositivesMultiClass(X_test, y_test, pred)
mislabeled = MislabeledMulitiClass(X_test,y_test,pred)

print(f"\nFP-> {len(false_positives)} \nFN-> {len(false_negatives)} \nMislabeled-> {len(mislabeled)}")

Logistic Regression Classifier
Accuracy=  0.9434439498178875
Precision=  N/A
Recall=  N/A

FP-> 75 
FN-> 479 
Mislabeled-> 5


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [108]:
# RANDOM FORREST
rfm = RandomForestClassifier().fit(X_train, y_train)
pred = rfm.predict(X_test)
acc = calcAccuracy(y_test, pred)
#recall = recall_score(y_test, pred)
#precision = precision_score(y_test, pred)
PrintResults("Random Forrest", acc, "N/A", "N/A")

false_negatives = FalseNegativesMultiClass(X_test, y_test, pred)
false_positives = FalsePositivesMultiClass(X_test, y_test, pred)
mislabeled = MislabeledMultiClass(X_test,y_test,pred)

print(f"\nFP-> {len(false_positives)} \nFN-> {len(false_negatives)} \nMislabeled-> {len(mislabeled)}")

Random Forrest Classifier
Accuracy=  0.9531566167543505
Precision=  N/A
Recall=  N/A

FP-> 126 
FN-> 332 
Mislabeled-> 5


In [109]:
# NAIVE BAYS
nb = GaussianNB().fit(X_train, y_train)
pred = nb.predict(X_test)
acc = calcAccuracy(y_test, pred)
#recall = recall_score(y_test, pred)
#precision = precision_score(y_test, pred)
PrintResults("Naive Bays", acc, "N/A", "N/A")

false_negatives = FalseNegativesMultiClass(X_test, y_test, pred)
false_positives = FalsePositivesMultiClass(X_test, y_test, pred)
mislabeled = MislabeledMulitiClass(X_test,y_test,pred)

print(f"\nFP-> {len(false_positives)} \nFN-> {len(false_negatives)} \n Mislabeled-> {len(mislabeled)}")

Naive Bays Classifier
Accuracy=  0.8475313638203157
Precision=  N/A
Recall=  N/A

FP-> 1176 
FN-> 263 
 Mislabeled-> 68


In [110]:
clf = MLPClassifier(solver='sgd', alpha=1e-5,
                     hidden_layer_sizes=(5, 100), learning_rate_init = 0.15, random_state=1)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print("class model performance: " + str(accuracy_score(y_test, pred)*100) + "%")
print("class model recall score: " + str(recall_score(y_test, pred, average = 'micro')*100) + "%")

class model performance: 95.22460542290571%
class model recall score: 95.22460542290571%


In [15]:
# Try/Catch for recognizing if input is json or not

#data_dict = list(map(lambda x : ast.literal_eval(x), data))
for x in dataset[:5]:
    print(x)

for y in dataset:
    x = None
    try:
        x = ast.literal_eval(y[0])
        #print(f"It didn't break! {x}")
    except (ValueError, SyntaxError):
        z = True
        #print(f"It broke:( {y}") 



In [ ]:
a = np.array([1,2,3,4,5,])
print(a)
print(a.reshape(1,-1))